In [2]:
import multiprocessing
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt
import re
from astropy.io import fits
from tqdm import tqdm
import os
import numpy as np
from scipy import interpolate
import spectres

## RestFrame

In [ ]:
# Specify the directory path
directory_path = ...
save_directory_path = ...
resampled_wavelength = np.linspace(
    np.log10(3800),
    np.log10(8400),
    4000
)

In [7]:
def save(i):
    # List all files and folders in the directory
    plates = os.listdir(directory_path)

    directory_path_plate = directory_path + plates[i]

    filenames = os.listdir(directory_path)
    
    pattern = r"spec-(\d+)-(\d+)-(\d+)\.fits"
    
    for filename in filenames: 
        # Search for the pattern in the filename
        match = re.search(pattern, filename)

        if match:
            # Extract the three numbers from the matched groups
            plate = int( match.group(1) )
            mjd = int( match.group(2) )
            fiber = int( match.group(3) )
        else:
            print('not match',filename)
            
        file_path = directory_path_plate + filename

        hdulist = fits.open(file_path)
        redshift = hdulist[2].data['Z'][0]
        if redshift < 0:
            redshift = 0
        elif redshift > 0.1:
            continue
    
        save_path_spec = save_directory_path + f"/plate{plate}/spec-{plate}-{mjd}-{fiber}.npy"
        save_path_error = save_directory_path + f"/plate{plate}/error-{plate}-{mjd}-{fiber}.npy"
        save_path_mask = save_directory_path + f"/plate{plate}/mask-{plate}-{mjd}-{fiber}.npy"
        
        if os.path.exists(save_path_spec) and os.path.exists(save_path_error):
            continue
        
        galaxy_class = hdulist[2].data['class'][0]
        if not galaxy_class == 'GALAXY':
            continue
        
        # load data
        flux = hdulist[1].data['flux']
        ivar = hdulist[1].data['ivar']
        wave = hdulist[1].data['loglam']
        # sky = hdulist[1].data['sky']
        # and_mask = hdulist[1].data['and_mask']
        # or_mask = hdulist[1].data['or_mask']
        # model_flux = hdulist[1].data['model']
        # Find indices of large error points
        
        try:
            ivar[ivar==0] = ivar[ivar!=0].min() 
        except:
            pass

        rest_wavelength = np.log10( 10**wave / ( redshift + 1 ) )
        error = np.sqrt(1.0 / ivar)


        # Resample the spectrum to 4000 data points in equal intervals using spectres
        resampled_flux = spectres.spectres(
            resampled_wavelength,
            rest_wavelength,  # The original wavelength array
            flux,  # The original flux array
            spec_errs=error,  # Optional: the original error array
            fill=np.nan,  # Optional: the value to fill extrapolated data points with.,
            verbose=False,
        )
        
        new_flux = resampled_flux[0]
        new_error = resampled_flux[1]

        # Define a threshold for large errors
        error_threshold = np.nanpercentile(error, 95)  # Adjust the percentile value as needed

        # Find indices of large error points
        mask = compute_mask(error,large_err=error_threshold,spectrum=flux)

        np.save(save_path_spec,new_flux)
        np.save(save_path_error,new_error)
        np.save(save_path_mask,mask)

In [ ]:
# Create a ThreadPoolExecutor
import concurrent.futures
with concurrent.futures.ThreadPoolExecutor(max_workers=48) as executor:
    # Use list comprehension to create a list of tasks to execute
    # Each task is a call to download_image with an index and a pair of coordinates
    tasks = [ executor.submit(save, i) for i in range(0,len(plates)) ]

    # Wait for all tasks to complete
    for future in concurrent.futures.as_completed(tasks):
        # If the task raised an exception, it will be re-raised here
        future.result()

/tmp/ipykernel_42825/3573525817.py:69: RuntimeWarning: divide by zero encountered in divide
  error = np.sqrt(1.0 / ivar)
/home/daikiiwasaki/yes/envs/torch2_env/lib/python3.10/site-packages/numpy/lib/function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
